### 0. Setup

In [1]:
import openai
from secret_keys import OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY
from utils import complete_prompt, count_tokens
import fitz
import pandas as pd
from io import StringIO
from IPython.display import display

In [7]:
NVISO_REPORT = "data/sample_reports/coronalert.pdf"
OFFSEC_REPORT = "data/sample_reports/sample_pentest_report.pdf"
ASVS_CATEGORIES_STR = """
- V1: Architecture, Design and Threat Modeling Requirements
- V2: Authentication Verification Requirements
- V3: Session Management Verification Requirements
- V4: Access Control Verification Requirements
- V5: Validation, Sanitization and Encoding Verification Requirements
- V6: Stored Cryptography Verification Requirements
- V7: Error Handling and Logging Verification Requirements
- V8: Data Protection Verification Requirements
- V9: Communications Verification Requirements
- V10: Malicious Code Verification Requirements
- V11: Business Logic Verification Requirements
- V12: File and Resources Verification Requirements
- V13: API and Web Service Verification Requirements
- V14: Configuration Verification Requirements
"""

In [8]:
def extract_all_text(pdf_location: str) -> str:
    """Extract all text from a PDF file."""
    doc = fitz.open(pdf_location)
    pdf_text = [page.get_text().strip().replace("\n", " ") for page in doc]
    return " ".join(pdf_text)

def fit_prompt_in_context(prompt: str, context_length: int = 3800) -> str:
    """
    Recursively shorten the prompt untill it fits the context window.
    Maximum context length of gpt-3.5-turbo is 4097 tokens.
    We assume findings are described later in the report, as opposed to earlier.
    """
    nr_tokens = count_tokens(prompt)
    if nr_tokens > context_length:
        return fit_prompt_in_context(prompt[250:])
    else:
        return prompt

def get_finding_extraction_prompt(report_text: str) -> str:
    prompt = f"""{report_text}
    
Given the above pentesting report, create a CSV of all security findings described in the report.
The CSV fields are:
- finding ID
- finding name
- finding risk

Findings:
"""
    prompt = fit_prompt_in_context(prompt)
    return prompt

def get_ASVS_classification_prompt(findings_csv: str) -> str:
    prompt = f"""CSV of all security findings: 
{findings_csv}

ASVS categories: {ASVS_CATEGORIES_STR}

Add a new field to the security findings CSV called "ASVS category", which categorizes the identified security findings into the correct ASVS category.
"""
    return prompt

# 1. Extract findings from multiple pentest reports

In [9]:
SYSTEM_ROLE = "You are a cybersecurity analyst processing pentesting reports."

In [10]:
reports = [NVISO_REPORT, OFFSEC_REPORT]
reports_text = [extract_all_text(report_loc) for report_loc in reports]
prompts = [get_finding_extraction_prompt(report_text) for report_text in reports_text]
responses = [complete_prompt(prompt=prompt, system_role=SYSTEM_ROLE) for prompt in prompts]

In [11]:
dfs = [pd.read_csv(StringIO(resp), sep=",", header=0)for resp in responses]
all_findings_csv = pd.concat(dfs)

In [12]:
display(all_findings_csv)


,finding ID,finding name,finding risk
0,A01,Valid TEKs marked as invalid,High
1,A02,Uploading of real TEKs can be identified in en...,Medium
2,A03,Positive result can be deduced from encrypted ...,Medium
3,A04,Application does not remove test result after ...,Medium
4,A05,Weak SSL/TLS configuration,Low
5,A06,Delivery of encrypted test result can be ident...,Low
6,A07,Submission of fake TEKs limited to 6hr window ...,Low
7,IAM01,Root user without MFA enabled,High
8,IAM02,IAM policies allowing full administrative priv...,Medium
9,IAM03,AWS API calls from service accounts or IAM use...,Medium


# 2. Classify findings according to ASVS categories

In [14]:
prompt = get_ASVS_classification_prompt(all_findings_csv.to_csv(index=False))
print(prompt)

CSV of all security findings: 
finding ID,finding name,finding risk
A01,Valid TEKs marked as invalid,High
A02,Uploading of real TEKs can be identified in encrypted traffic due to incorrect padding,Medium
A03,Positive result can be deduced from encrypted traffic,Medium
A04,Application does not remove test result after upload of TEKs,Medium
A05,Weak SSL/TLS configuration,Low
A06,Delivery of encrypted test result can be identified in rare cases,Low
A07,Submission of fake TEKs limited to 6hr window each day,Low
IAM01,Root user without MFA enabled,High
IAM02,IAM policies allowing full administrative privileges are created,Medium
IAM03,AWS API calls from service accounts or IAM users are not restricted,Medium
BC01,AWS VPN not redundant,Low
DP01,S3 security controls disabled,Low
NET03,Advanced DDoS protection is not enabled,Low
CM01,Sensitive data exposed to EC2 instance user data,Low
1,Default or Weak Credentials,High
2,Password Reuse,High
3,Shared Local Administrator Password,High
4,Patch M

In [15]:
response = complete_prompt(prompt=prompt, system_role=SYSTEM_ROLE)

In [16]:
display(pd.read_csv(StringIO(response)))

,finding ID,finding name,finding risk,ASVS category
0,A01,Valid TEKs marked as invalid,High,V4
1,A02,Uploading of real TEKs can be identified in en...,Medium,V5
2,A03,Positive result can be deduced from encrypted ...,Medium,V5
3,A04,Application does not remove test result after ...,Medium,V7
4,A05,Weak SSL/TLS configuration,Low,V9
5,A06,Delivery of encrypted test result can be ident...,Low,V9
6,A07,Submission of fake TEKs limited to 6hr window ...,Low,V4
7,IAM01,Root user without MFA enabled,High,V2
8,IAM02,IAM policies allowing full administrative priv...,Medium,V4
9,IAM03,AWS API calls from service accounts or IAM use...,Medium,V4
